In [ ]:
import sys
import os
import torch

os.environ['CUDA_DEVICE_ORDER']='PCI_BUS_ID'
os.environ['CUDA_VISIBLE_DEVICES']='1'

import numpy as np
import plotly.graph_objects as go
import matplotlib.pyplot as plt

In [ ]:
def toDisplay(x, target_dim = 2):
    while(x.dim() > target_dim):
        x = x[0]
    return x.detach().cpu().numpy()

In [ ]:
import sys
sys.path.insert(1, '/home/exx/Documents/equivariant_pose_graph/python')
from equivariant_pose_graph.models.transformer_flow import ResidualFlow, ResidualFlow_V1
from equivariant_pose_graph.training.flow_equivariance_training_module import EquivarianceTrainingModule
 
# from ndf_robot.eval.test_trained_model_place import load_data
from equivariant_pose_graph.utils.display import scatter3d, quiver3d
from pytorch3d.transforms import Transform3d

In [ ]:
def plot(points_action, points_anchor):
    colors = [
    '#1f77b4',  # muted blue
    '#ff7f0e',  # safety orange
    '#2ca02c',  # cooked asparagus green
    '#d62728',  # brick red
    '#9467bd',  # muted purple
    '#8c564b',  # chestnut brown
    '#e377c2',  # raspberry yogurt pink
    '#7f7f7f',  # middle gray
    '#bcbd22',  # curry yellow-green
    '#17becf'   # blue-teal
]
    skip = 1
    points_action_dp = toDisplay(points_action)
    points_anchor_dp = toDisplay(points_anchor)
    go_data=[
        go.Scatter3d(x=points_action_dp[::skip,0], y=points_action_dp[::skip,1], z=points_action_dp[::skip,2], 
                     mode='markers', marker=dict(size=1, color=colors[0],
                     symbol='circle')),
        go.Scatter3d(x=points_anchor_dp[::skip,0], y=points_anchor_dp[::skip,1], z=points_anchor_dp[::skip,2], 
                     mode='markers', marker=dict(size=1, color=colors[1],
                     symbol='circle')),
    ]
    layout = go.Layout(
        scene=dict(
            aspectmode='data'
        )
    )

    fig = go.Figure(data=go_data, layout=layout)
    fig.show()

In [ ]:
network = ResidualFlow()

model = EquivarianceTrainingModule(
    network)
model.cuda()
# checkpoint_file = "/home/exx/media/DataDrive/singularity_chuerp/equiv_pgraph_logs/train_test_mr_dcpflow_residual0/equiv_dcpflow/version_9/checkpoints/epoch=410-step=51375.ckpt"
# checkpoint_file = "/home/exx/media/DataDrive/singularity_chuerp/equiv_pgraph_logs/train_test_mr_dcpflow_residual0_attn_trans0.1_overfit/equiv_dcpflow/version_3/saved_checkpoints/epoch=30-step=3875.ckpt"
# checkpoint_file = "/home/exx/media/DataDrive/singularity_chuerp/equiv_pgraph_logs/train_test_mr_dcpflow_residual1_attn/equiv_dcpflow/version_1/checkpoints/epoch=754-step=94375.ckpt"
checkpoint_file = '/home/exx/media/DataDrive/singularity_chuerp/equiv_pgraph_logs/train_test_mr_dcpflow_residual0_attn_trans0.1_ovefit/equiv_dcpflow/version_26/checkpoints/epoch=2-step=375.ckpt'
model.load_state_dict(torch.load(checkpoint_file)['state_dict'])
 
##
place_model = model# .eval()

In [ ]:
pred_T = np.load('/home/exx/Documents/equivariant_pose_graph/transformations/display_pointclouds.npz', allow_pickle=True)
pred_T_action = torch.from_numpy(pred_T['pred_T_action'])
pred_T_action_transform = Transform3d(matrix=pred_T_action)
 
pred_T_anchor = torch.from_numpy(pred_T['pred_T_anchor'])
pred_T_anchor_transform = Transform3d(matrix=pred_T_anchor)
points_trans_action = torch.from_numpy(pred_T['points_trans_action'])
points_trans_anchor = torch.from_numpy(pred_T['points_trans_anchor'])
action_transformed = torch.from_numpy(pred_T['action_transformed'])
anchor_transformed = torch.from_numpy(pred_T['anchor_transformed'])
img_action = torch.from_numpy(pred_T['img_action'])

our_action_transformed = pred_T_action_transform.transform_points(points_trans_action)
our_anchor_transformed = pred_T_anchor_transform.transform_points(points_trans_anchor)

In [ ]:
%matplotlib inline
plt.imshow(img_action)
plt.show()

In [ ]:
plot(our_action_transformed[0], points_trans_anchor[0])

In [ ]:
plot(our_anchor_transformed[0], points_trans_action[0])

In [ ]:
pred_T_action, pred_T_anchor, pred_T  = place_model(points_trans_action.cuda(), points_trans_anchor.cuda())
our_action_transformed = pred_T_action.transform_points(points_trans_action.cuda())

In [ ]:
torch.allclose(our_action_transformed.detach().cpu(), action_transformed, atol=1e-3)

In [ ]:
plot(our_action_transformed[0], points_trans_anchor[0])

In [ ]:
stop

In [ ]:

# T = np.load('/home/exx/Documents/equivariant_pose_graph/transformations/mug_on_rack.npz', allow_pickle=True)
# T0 = T['T0_list'][0]
# T1 = T['T1_list'][0]
# print(type(T0))
# print(T0.shape)
# T0 = Transform3d(matrix=T0)
# T1 = Transform3d(matrix=T1)


In [ ]:
# pred_T = np.load('/home/exx/Documents/equivariant_pose_graph/transformations/display_pointclouds.npz', allow_pickle=True)
# pred_T_action = torch.from_numpy(pred_T['pred_T_action'])
# pred_T_action_transform = Transform3d(matrix=pred_T_action)
 
# points_trans_action = torch.from_numpy(pred_T['points_trans_action'])
# points_trans_anchor = torch.from_numpy(pred_T['points_trans_anchor'])

 

# action_transformed = torch.from_numpy(pred_T['action_transformed'])
# img_action = torch.from_numpy(pred_T['img_action'])

# our_action_transformed = pred_T_action_transform.transform_points(points_trans_action)
# torch.allclose(our_action_transformed, action_transformed, atol=1e-3)
# %matplotlib inline
# plt.imshow(img_action)
# plt.show()
# plot(our_action_transformed[0], points_trans_anchor[0])

In [ ]:
torch.allclose(our_action_transformed, action_transformed, atol=1e-3)

In [ ]:
%matplotlib inline
plt.imshow(img_action)
plt.show()

In [ ]:
plot(our_action_transformed[0], points_trans_anchor[0])

In [ ]:
# pred_T = np.load('/home/exx/Documents/equivariant_pose_graph/transformations/mug_on_rack_predT.npz', allow_pickle=True)
# pred_T_action_1 = torch.from_numpy(pred_T['pred_T_action'])
# pred_T_anchor_1 = torch.from_numpy(pred_T['pred_T_anchor'])

# pred_T_action_predT = Transform3d(matrix=pred_T_action_1)
# pred_T_anchor_predT= Transform3d(matrix=pred_T_anchor_1)
 
# points_trans_action_predT = torch.from_numpy(pred_T['points_trans_action'])
# points_trans_anchor_predT  = torch.from_numpy(pred_T['points_trans_anchor'])

# points_action_predT  = torch.from_numpy(pred_T['points_action'])
# points_anchor_predT  = torch.from_numpy(pred_T['points_anchor'])

# action_transformed_predT = torch.from_numpy(pred_T['action_transformed'])
# anchor_transformed_predT = torch.from_numpy(pred_T['anchor_transformed'])

# APP_action_transformed_predT = pred_T_action_predT.transform_points(points_trans_action_predT)
# APP_anchor_transformed_predT = pred_T_anchor_predT.transform_points(points_trans_anchor_predT)
 

In [ ]:
# display_pointclouds = np.load('/home/exx/Documents/equivariant_pose_graph/transformations/display_pointclouds.npz', allow_pickle=True)
# action_transformed = torch.from_numpy(display_pointclouds['action_transformed'])
# points_trans_disp_anchor = torch.from_numpy(display_pointclouds['points_trans_disp_anchor'])
# anchor_transformed = torch.from_numpy(display_pointclouds['anchor_transformed'])
# points_trans_disp_action = torch.from_numpy(display_pointclouds['points_trans_disp_action'])
# img_action = display_pointclouds['img_action']
# img_anchor = display_pointclouds['img_anchor']

# points_trans_action  = torch.from_numpy(display_pointclouds['points_trans_action'])
# points_trans_anchor   = torch.from_numpy(display_pointclouds['points_trans_anchor'])

# points_action = torch.from_numpy(display_pointclouds['points_action'])
# points_anchor   = torch.from_numpy(display_pointclouds['points_anchor'])
# pred_T_action = torch.from_numpy(display_pointclouds['pred_T_action'])
# pred_T_anchor = torch.from_numpy(display_pointclouds['pred_T_anchor'])

# pred_T_action = Transform3d(matrix=pred_T_action)
# pred_T_anchor= Transform3d(matrix=pred_T_anchor)
# action_transformed_diff = pred_T_action.transform_points(points_trans_action)
# anchor_transformed_diff = pred_T_anchor.transform_points(points_trans_anchor)
 
# print(points_trans_disp_action.shape)
# print(action_transformed.shape)

In [ ]:
 torch.allclose(action_transformed_diff, action_transformed)

In [ ]:
torch.allclose(action_transformed_predT,action_transformed)

In [ ]:
torch.allclose(points_trans_action_predT, points_trans_action)

In [ ]:
print(points_trans_anchor_predT[0].shape)
print(points_trans_disp_anchor.shape)

In [ ]:
torch.allclose(points_trans_anchor_predT[0],points_trans_disp_anchor)

In [ ]:
torch.allclose(action_transformed_predT[0],action_transformed)

In [ ]:
%matplotlib inline
plt.imshow(img_action)
plt.show()

In [ ]:
%matplotlib inline
res = scatter3d(
            xs = [action_transformed, points_trans_disp_anchor],
            cs = ['b', 'r'],
        )
plt.imshow(res)
plt.show()

In [ ]:
plot(action_transformed, points_trans_disp_anchor)

In [ ]:
plot(anchor_transformed, points_trans_disp_action)

In [ ]:
from pathlib import Path
import torch.nn.functional as F

# num_classes = 3
# data_idx = 0
# cloud_types = ['init', 'pre_grasp', 'post_grasp', 'teleport', 'post_place', 'final']
# cloud_type = 'teleport'
# # data_path = Path('/home/bokorn/src/ndf_robot/notebooks')
# # point_data = np.load(data_path / f'{data_idx}_obj_points.npz')
# # data_path = Path('/home/exx/Documents/ndf_robot/train_new_data_3/renders')
# data_path = Path('/home/exx/Documents/ndf_robot/src/ndf_robot/debug_place_shapeid_34ae0b61b0d8aaf2d7b20fded0142d7a')
# point_data = np.load(data_path / f'{data_idx}_{cloud_type}_obj_points.npz', allow_pickle = True)

# points_raw_np = point_data['clouds']
# classes_raw_np = point_data['classes']
  
# class_0 = 0
# class_1 = 1
# class_2 = 2
# num_classes = 2
# mask = np.isin(classes_raw_np, [class_0, class_1])
# points_np = points_raw_np[mask].copy()
# classes_np = np.where(classes_raw_np[mask] == class_0, 0, 1)
 
# print(classes_np.shape)

# # points_np = points_np - points_np[classes_np==class_1].mean(axis=0)
# points_np = points_np - points_np[classes_np==class_0].mean(axis=0)

# points = torch.from_numpy(points_np).unsqueeze(0).float().cuda()
# class_ids = torch.from_numpy(classes_np).unsqueeze(0).long().cuda()
# classes = F.one_hot(class_ids, num_classes=num_classes)
# points = points[:,::2]
# classes = classes[:,::2]
# # object_points = []
# # for j in range(num_classes):
# #     object_points.append(points[class_ids==j,:])
# # colors_np = point_data['colors']
# print(points_np.shape)


In [ ]:
# demo
point_data = np.load('/home/exx/Documents/ndf_robot/train_data_3/renders/0_teleport_obj_points.npz', allow_pickle = True)
points_raw_np = point_data['clouds']
classes_raw_np = point_data['classes']

class_0 = 0
class_1 = 1
class_2 = 2
num_classes = 2
mask = np.isin(classes_raw_np, [class_0, class_1])
points_np = points_raw_np[mask].copy()
classes_np = np.where(classes_raw_np[mask] == class_0, 0, 1)
 
print(classes_np.shape)

# points_np = points_np - points_np[classes_np==class_1].mean(axis=0)
# points_np = points_np - points_np[classes_np==class_0].mean(axis=0)

points = torch.from_numpy(points_np).unsqueeze(0).float().cuda()
class_ids = torch.from_numpy(classes_np).unsqueeze(0).long().cuda()
classes = F.one_hot(class_ids, num_classes=num_classes)
points = points[:,::2]
classes = classes[:,::2]

skip = 1
 
go_data=[
    go.Scatter3d(x=points_np[::skip,0], y=points_np[::skip,1], z=points_np[::skip,2], 
                 mode='markers', marker=dict(size=1, color=classes_np[::skip],
                 symbol='circle')),
]
layout = go.Layout(
    scene=dict(
        aspectmode='data'
    )
)

fig = go.Figure(data=go_data, layout=layout)
fig.show()

In [ ]:
plot(action_transformed[0], points_anchor[0])

In [ ]:
points_action_pre, points_anchor_pre = load_data(num_points=1024, clouds = point_data['clouds'] ,classes = point_data['classes'], action_class= 0, anchor_class= 1)
T0 = T0.to(device =  points_action_pre.device)
T1 = T1.to(device =  points_anchor_pre.device)
points_action = T0.transform_points(points_action_pre)
points_anchor = T1.transform_points(points_anchor_pre)
 
pred_T_action, pred_T_anchor, pred_T  = place_model(points_action, points_anchor)

pred_T_action_points_action = pred_T_action.transform_points(points_action)
pred_T_action_points_anchor = pred_T_anchor.transform_points(points_anchor)
# pred_T_points_action = pred_T.transform_points(points_action)
# pred_T_action_inverse_points_action = pred_T_anchor.inverse().transform_points(points_action)

In [ ]:
plot(points_action_pre, points_anchor_pre)

In [ ]:
plot(points_action, points_anchor)

In [ ]:
plot(pred_T_action_points_action, points_anchor)

In [ ]:
plot(points_action, points_anchor)

In [ ]:
plot(pred_T_action_points_action, points_anchor)

In [ ]:
plot(points_action, pred_T_action_points_anchor)

In [ ]:
points_action, points_anchor = load_data(num_points=1024, clouds = point_data['clouds'] ,classes = point_data['classes'], action_class= 0, anchor_class= 1)
pred_T_action, pred_T_anchor, pred_T  = place_model(points_action, points_anchor)
 
pred_T_action_points_action = pred_T_action.transform_points(points_action)
for i in range(20):
    pred_T_action, pred_T_anchor, pred_T  = place_model(pred_T_action_points_action, points_anchor)
    pred_T_action_points_action = pred_T_action.transform_points(pred_T_action_points_action)
    plot(pred_T_action_points_action, points_anchor)